# Deep Colorization

## Overview

This notebook creates a model that is able to colorize images to a certain extent, which combines a Fast deep Convolutional Neural Network trained from scratch with high-level features extracted from the MobileNet pre-trained model. This encoder-decoder model can process images of any size and aspect ratio. The training of this model is done on 60K images of MS-COCO dataset. How this model performs in coloring images are also showing in result section.

This notebook's work is inspired from https://github.com/titu1994/keras-mobile-colorizer which is also transfer to ipynb notebook too [[link]()].

## Introduction

There is something uniquely and powerfully satisfying about the simple act of adding color to black and white imagery. Moreover this coloring of gray-scale images can have a big impact in a wide variety of domains, for instance, re-master of historical images, dormant memories or expressing artistic creativity and improvement of surveillance feeds.

The information content of a gray-scale image is rather limited, thus adding the color components can provide more insights about its semantics. In the context of deep learning, models such as Inception [[ref]()], VGG [[ref]()] and others are usually trained using colored image datasets. When applying these networks on grayscale images, a prior colorization step can help improve the results. However, designing and implementing an effective and reliable system that automates this process still remains nowadays as a challenging task.

In this regard, below is the proposed model that is able to colorize images to a certain extent, combining a DCNN [[ref]()] architecture which utilizes a U-Net inspired model conditioned on MobileNet class features to generate a mapping from Grayscale to Color image. This work is based on the https://github.com/titu1994/keras-mobile-colorizer and https://github.com/baldassarreFe/deep-koalarization [[research paper](https://arxiv.org/pdf/1712.03400.pdf)].

### My contribution are as follows: 
  
1. Using fast algorithms for CNNs based on the minimal filtering algorithms pioneered by Winograd [[research paper](https://arxiv.org/abs/1509.09308)]
2. Analysis and intuition behind a colorization architecture based on CNNs.

## Background
Below are the few components that are used in the architecture of this model. Basic introduction of these components are given and more information you see the links provided in each section.


### U-Net: Convolutional Networks

The U-Net architecture is illustrated in below shown figure. It consists of a contracting path (left side) and an expansive path (right side). The architecture is that in the upsampling part, it has a large number of feature channels, which allow the network to propagate context information to higher resolution layers. As a consequence,
the expansive path is more or less symmetric to the contracting path, and yields a u-shaped architecture.

The contracting path follows the typical architecture of a convolutional network. It consists of the repeated application of two convolutions mostly 3x3 (unpadded convolutions), each followed by a rectified linear unit (ReLU) and a max pooling mostly 2x2 operation with stride 2 for downsampling. At each downsampling step we double the number of feature channels. Every step in the expansive path consists of an upsampling of the feature map followed by a convolution mostly 2x2 (“up-convolution”) that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, and two convolutions mostly 2x2 , each followed by a ReLU. The cropping is necessary due to the loss of border pixels in every convolution. At the final layer a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. In total the below network has 23 convolutional layers.


![U-Net Architecture](https://raw.githubusercontent.com/krypten/MobileDeepColorization/master/docs/images/u_net_architecture.png)



### MobileNet  
  
MobileNets are an efficent class of convolutional neural network. The main difference between the MobileNet architecture and a “traditional” CNN’s is instead of a single 3x3 convolution layer followed by batch norm and ReLU, MobileNets split the convolution into a 3x3 depthwise convolution and a 1×1 convolution called a pointwise convolution.


MobileNets introduce two simple global hyperparameters that efficiently trade off between latency and
accuracy : width multiplier and resolution multiplier. These hyper-parameters allow the model builder to choose the right sized model for their application based on the constraints of the problem. The width multiplier allows us to thin the network, while the resolution multiplier changes the input dimensions of the image, reducing the internal representation at every layer.

To learn more about how MobileNets work, read the [research paper](https://arxiv.org/pdf/1704.04861.pdf).

![MobileNet convolution](https://raw.githubusercontent.com/krypten/MobileDeepColorization/master/docs/images/mobilenet_architecture.png)

## Architecture
...


![Deep Colorization Architecture](https://raw.githubusercontent.com/krypten/MobileDeepColorization/master/docs/images/deep_colorization_architecture.png)

## Experimentation

##### Install required modules

In [1]:
%%capture capt

# Install pip packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikit-image
!{sys.executable} -m pip install tensorflow

print(capt.stderr) # Only display the errors and suppressed other output

NameError: name 'capt' is not defined

##### Import required modules

In [2]:
## Import required modules
import keras
from keras.layers import Conv2D, Input, Reshape, RepeatVector, concatenate, UpSampling2D, Flatten, Conv2DTranspose
from keras.models import Model

from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard

from keras.losses import mean_squared_error
from keras.optimizers import Adam

import numpy as np

from skimage.color import rgb2gray
from skimage.transform import resize

import tensorflow as tf
import utils

weights_file_name = 'weights/mobilenet_model_improved.h5'

# Fetch the util file and import it
# TODO

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'utils'

### Check the Version of TensorFlow and Access to GPU¶

In [ ]:


# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

##### Get the Data
Run the following cell to download the dataset.

In [ ]:
# Configure Data for use
util.configure_tensforflow()

# Load preprocessed data
util.load_preprocessed_tfrecord_data()

### Model
...

### Hyperparameters

In [ ]:
### Hyperparameters
batch_size = 100
epochs = 100
image_size = 256
# nb_train_images = 60000 # there are 82783 images in MS-COCO, set this to how many samples you want to train on.

In [ ]:
from keras.datasets import cifar10

# Load data
(y_train, _), (y_test, _) = cifar10.load_data()
x_train = np.expand_dims(rgb2gray(y_train), axis=3)
x_test = np.expand_dims(rgb2gray(y_test), axis=3)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [ ]:
img_height, img_width = x_train.shape[1], x_train.shape[2]
print("Image Height : {}, Weight : {}".format(img_height, img_width))

image_size = img_height

### Metrics and Loss function


In [ ]:
mse_weight = 1.0 #1e-3

# set these to zeros to prevent learning
perceptual_weight = 1. / (2. * 128. * 128.) # scaling factor
attention_weight = 1.0 # 1.0


# shows the minimum value of the AB channels
def y_true_min(yt, yp):
    return K.min(yt)


# shows the maximum value of the RGB AB channels
def y_true_max(yt, yp):
    return K.max(yt)


# shows the minimum value of the predicted AB channels
def y_pred_min(yt, yp):
    return K.min(yp)


# shows the maximum value of the predicted AB channels
def y_pred_max(yt, yp):
    return K.max(yp)


def gram_matrix(x):
    assert K.ndim(x) == 4

    with K.name_scope('gram_matrix'):
        if K.image_data_format() == "channels_first":
            batch, channels, width, height = K.int_shape(x)
            features = K.batch_flatten(x)
        else:
            batch, width, height, channels = K.int_shape(x)
            features = K.batch_flatten(K.permute_dimensions(x, (0, 3, 1, 2)))

        gram = K.dot(features, K.transpose(features)) # / (channels * width * height)
    return gram


def l2_norm(x):
    return K.sqrt(K.sum(K.square(x)))


def attention_vector(x):
    if K.image_data_format() == "channels_first":
        batch, channels, width, height = K.int_shape(x)
        filters = K.batch_flatten(K.permute_dimensions(x, (1, 0, 2, 3)))  # (channels, batch*width*height)
    else:
        batch, width, height, channels = K.int_shape(x)
        filters = K.batch_flatten(K.permute_dimensions(x, (3, 0, 1, 2)))  # (channels, batch*width*height)

    filters = K.mean(K.square(filters), axis=0)  # (batch*width*height,)
    filters = filters / l2_norm(filters)  # (batch*width*height,)
    return filters


def total_loss(y_true, y_pred):
    mse_loss = mse_weight * mean_squared_error(y_true, y_pred)
    perceptual_loss = perceptual_weight * K.sum(K.square(gram_matrix(y_true) - gram_matrix(y_pred)))
    attention_loss = attention_weight * l2_norm(attention_vector(y_true) - attention_vector(y_pred))

    return mse_loss + perceptual_loss + attention_loss

### Build Model


In [ ]:
def build_mobilenet_model(img_size, lr=1e-3):
    '''
    Creates a Colorizer model. Note the difference from the report
    - https://github.com/baldassarreFe/deep-koalarization/blob/master/report.pdf
    I use a long skip connection network to speed up convergence and
    boost the output quality.
    '''
    ## Encoder Model
    encoder_input = Input(shape=(img_size, img_size, 1,))
    encoder1 = Conv2D(64, (3, 3), padding='same', activation='relu', strides=(2, 2))(encoder_input)
    encoder = Conv2D(128, (3, 3), padding='same', activation='relu')(encoder1)
    encoder2 = Conv2D(128, (3, 3), padding='same', activation='relu', strides=(2, 2))(encoder)
    encoder = Conv2D(256, (3, 3), padding='same', activation='relu')(encoder2)
    encoder = Conv2D(256, (3, 3), padding='same', activation='relu', strides=(2, 2))(encoder)
    encoder = Conv2D(512, (3, 3), padding='same', activation='relu')(encoder)
    encoder = Conv2D(512, (3, 3), padding='same', activation='relu')(encoder)
    encoder = Conv2D(256, (3, 3), padding='same', activation='relu')(encoder)

    ## Input Fusion
    # Decide the image shape at runtime to allow prediction on
    # any size image, even if training is on 128x128
    batch, height, width, channels = K.int_shape(encoder)

    #mobilenet_features_ip = Input(shape=(1000,))
    #fusion = RepeatVector(height * width)(mobilenet_features_ip)
    #fusion = Reshape((height, width, 1000))(fusion)
    #fusion = concatenate([encoder, fusion], axis=-1)
    fusion = Conv2D(256, (1, 1), padding='same', activation='relu')(encoder)

    ## Decoder Model
    decoder = Conv2D(128, (3, 3), padding='same', activation='relu')(fusion)
    decoder = UpSampling2D()(decoder)
    #decoder = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu')(decoder)
    decoder = concatenate([decoder, encoder2], axis=-1)
    decoder = Conv2D(64, (3, 3), padding='same', activation='relu')(decoder)
    decoder = Conv2D(64, (3, 3), padding='same', activation='relu')(decoder)
    decoder = UpSampling2D()(decoder)
    #decoder = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', activation='relu')(decoder)
    decoder = concatenate([decoder, encoder1], axis=-1)
    decoder = Conv2D(32, (3, 3), padding='same', activation='relu')(decoder)
    decoder = Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh')(decoder)
    # decoder = Conv2D(2, (3, 3), padding='same', activation='tanh')(decoder)
    # decoder = UpSampling2D((2, 2))(decoder)

    model = Model([encoder_input], decoder, name='Colorizer')
    model.compile(optimizer=Adam(lr), loss=total_loss, metrics=[y_true_max,
                                                                y_true_min,
                                                                y_pred_max,
                                                                y_pred_min])

    print("Model built and compiled")
    return model

In [ ]:
# Model Summary
model = build_mobilenet_model(image_size, 1e-3)
model.summary()

## Training
...

In [ ]:
import os

# Continue training if weights are available
if os.path.exists(weights_file_name):
    model.load_weights(weights_file_name)

# Use Batchwise TensorBoard callback
tensorboard = TensorBoard(batch_size=batch_size)
checkpoint = ModelCheckpoint(weights_file_name, monitor='loss', verbose=1, save_best_only=True)
callbacks_list = [checkpoint, tensorboard]

# Train Network
print(x_train.shape)
print(y_train.shape)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=callbacks_list,
          verbose=1,
          validation_data=(x_test, y_test))

## Evaluation
Test the model against the test dataset.

In [ ]:
# Load the best weights
from keras.models import load_model
best_model = load_model(weights_file_name)

# Test the model
x_test, y_test = utils.prepare_input_image_batch(test_data, batch_size=batch_size)

score = best_model.evaluate(x_test, y_test, batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Show images for test data
To be added
# predictions = model.predict(x_test, batch_size, verbose=1)
#postprocess_output(x_test, predictions, image_size=image_size)

## Results 
Test the model against other images.

## Conclusions and Future Work
...